In [ ]:
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
# API_KEY = os.getenv("CONGRESS_API_KEY")
API_KEY = "cW09gbNMaffaTf3nQczNUKl7XZt5lgHYL40uDDMn" # cadens

BASE_URL = "https://api.congress.gov/v3"
endpoint = "/bill"
congresses = [i for i in range(106, 120)] # year 2000-2025
# congresses = [118]
congresses.reverse()
bill_types = ["s", "hjres", "sjres", "hr"] # house of reps, senate, house joint res, senate joint res
params = {"api_key" : API_KEY,
          "limit" : 1} # 250 is max

COMPLETED = []

congresses

In [ ]:
import json
import time

def check_for_rate_limit(r_json):
    if "error" in r_json and "code" in r_json["error"]:
        if r_json["error"]["code"] == "OVER_RATE_LIMIT":
            print("OVER_RATE_LIMIT: sleeping for 1800")
            time.sleep(1800) # sleep 30 min
            return True
    return False

# get counts for each bill type in a given congress
for c in congresses:
    if c in COMPLETED: # skip
        continue
    congress = f"/{c}"
    
    for b_t in bill_types:
        params["limit"] = 1 # min
        bills = {}
        bill_type = f"/{b_t}"
        url = f"{BASE_URL}{endpoint}{congress}{bill_type}"
        response = requests.get(url, params=params)
        r_json = json.loads(response.text)
        while check_for_rate_limit(r_json):
            response = requests.get(url, params=params)
            r_json = json.loads(response.text)
        total_bills_in_bill_type = r_json["pagination"]["count"]
        
        for i in range(1, total_bills_in_bill_type + 1):
            try:
                params["limit"] = 250 # max
                print(c, b_t, i, total_bills_in_bill_type)
                key = f"{c}_{b_t}_{i}"
                tmp_dict = {}

                actions_url = f"{BASE_URL}{endpoint}{congress}{bill_type}/{i}/actions"
                response = requests.get(actions_url, params=params)
                r_json = json.loads(response.text)
                while check_for_rate_limit(r_json):
                    response = requests.get(url, params=params)
                    r_json = json.loads(response.text)
                tmp_dict["actions"] = r_json
            except Exception as e:
                error_message = f"ERROR: {e}\nTYPE: {type(e)}"
                tmp_dict = {}
                tmp_dict["error"] = error_message
                
            bills[key] = tmp_dict
            
        with open(f"{c}_{b_t}_actions.json", "w") as f:
            json.dump(bills, f)